In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [21]:
df = pd.read_csv('results/ncaa_scores_2024-09-23.csv')

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Split score into home and away goals
df[['home_goals', 'away_goals']] = df['score'].str.split('-', expand=True)
df['home_goals'] = df['home_goals'].astype(int)
df['away_goals'] = df['away_goals'].astype(int)

# Initialize teams' performance indicators
teams = set(df['home_team']).union(df['away_team'])
team_stats = {team: {'home_goals_scored': 0, 'home_goals_conceded': 0,
                     'away_goals_scored': 0, 'away_goals_conceded': 0, 'matches': 0} for team in teams}

# Feature engineering
for i, row in df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    team_stats[home_team]['home_goals_scored'] += row['home_goals']
    team_stats[home_team]['home_goals_conceded'] += row['away_goals']
    team_stats[away_team]['away_goals_scored'] += row['away_goals']
    team_stats[away_team]['away_goals_conceded'] += row['home_goals']
    
    team_stats[home_team]['matches'] += 1
    team_stats[away_team]['matches'] += 1

# Create a feature DataFrame
team_features = []
for team, stats in team_stats.items():
    avg_home_goals = stats['home_goals_scored'] / stats['matches']
    avg_away_goals = stats['away_goals_scored'] / stats['matches']
    avg_goals_conceded = (stats['home_goals_conceded'] + stats['away_goals_conceded']) / stats['matches']
    team_features.append([team, avg_home_goals, avg_away_goals, avg_goals_conceded])

team_features_df = pd.DataFrame(team_features, columns=['team', 'avg_home_goals', 'avg_away_goals', 'avg_goals_conceded'])

# Let's assume the label we're predicting is the team's average goals scored
X = team_features_df[['avg_home_goals', 'avg_away_goals', 'avg_goals_conceded']]
y = team_features_df['avg_home_goals']  # Predicting home goals, for example

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')

Mean Squared Error: 0.0013234122023809544


In [12]:
teams = [
    "Pittsburgh",
    "Stanford",
    "North Carolina",
    "Ohio State",
    "Denver",
    "West Virginia",
    "Marshall",
    "Wisconsin",
    "Charlotte",
    "James Madison",
    "Western Michigan",
    "Clemson",
    "UC Santa Barbara",
    "Elon",
    "Seton Hall",
    "SMU",
    "Virginia Tech",
    "Providence",
    "Maryland",
    "Penn",
    "San Diego",
    "Hofstra",
    "VCU",
    "UConn",
    "Michigan"
]


In [23]:
# Split score into home and away goals
df[['home_goals', 'away_goals']] = df['score'].str.split('-', expand=True)
df['home_goals'] = df['home_goals'].astype(int)
df['away_goals'] = df['away_goals'].astype(int)

# Initialize teams' performance indicators
teams = set(df['home_team']).union(df['away_team'])
team_stats = {team: {'home_goals_scored': 0, 'home_goals_conceded': 0,
                     'away_goals_scored': 0, 'away_goals_conceded': 0, 'matches': 0} for team in teams}

# Feature engineering
for i, row in df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    team_stats[home_team]['home_goals_scored'] += row['home_goals']
    team_stats[home_team]['home_goals_conceded'] += row['away_goals']
    team_stats[away_team]['away_goals_scored'] += row['away_goals']
    team_stats[away_team]['away_goals_conceded'] += row['home_goals']
    
    team_stats[home_team]['matches'] += 1
    team_stats[away_team]['matches'] += 1

# Create a feature DataFrame
team_features = []
for team, stats in team_stats.items():
    avg_home_goals = stats['home_goals_scored'] / stats['matches']
    avg_away_goals = stats['away_goals_scored'] / stats['matches']
    avg_goals_conceded = (stats['home_goals_conceded'] + stats['away_goals_conceded']) / stats['matches']
    team_features.append([team, avg_home_goals, avg_away_goals, avg_goals_conceded])

team_features_df = pd.DataFrame(team_features, columns=['team', 'avg_home_goals', 'avg_away_goals', 'avg_goals_conceded'])

# Define the features (X) and label (y) for training
X = team_features_df[['avg_home_goals', 'avg_away_goals', 'avg_goals_conceded']]
y = team_features_df['avg_home_goals']  # We're predicting home goals here

# Train the model
model = RandomForestRegressor()
model.fit(X, y)

# Predict team abilities using the same features
team_features_df['predicted_ability'] = model.predict(X)

# Rank teams based on predicted ability (higher predicted ability = stronger team)
team_ranking = team_features_df[['team', 'predicted_ability']].sort_values(by='predicted_ability', ascending=False)

# Display the ranked teams
team_ranking.head(25)

,team,predicted_ability
132,South Carolina,2.717500
167,Wisconsin,2.634643
41,Duke,2.417063
141,NC State,2.291746
166,Brown,1.993889
84,Marshall,1.988905
142,West Virginia,1.967754
74,Gonzaga,1.904508
88,Western Mich.,1.894048
210,Stanford,1.886675


In [13]:
rankings = team_ranking.head(25)

In [20]:
import pandas as pd

# Sample lists
list_1 = rankings['team']

list_2 = teams

df = pd.DataFrame({
    'Team': list_1,
    'List 1 Order': range(1, len(list_1) + 1),
})

# Add the order for list 2, defaulting to 26 for teams not found
df['List 2 Order'] = df['Team'].apply(lambda x: list_2.index(x) + 1 if x in list_2 else 26)

# Calculate the order difference
df['Order Difference'] = df['List 1 Order'] - df['List 2 Order']

print(df)


                  Team  List 1 Order  List 2 Order  Order Difference
132     South Carolina             1            26               -25
167          Wisconsin             2             8                -6
41                Duke             3            26               -23
141           NC State             4            26               -22
166              Brown             5            26               -21
84            Marshall             6             7                -1
142      West Virginia             7             6                 1
74             Gonzaga             8            26               -18
88       Western Mich.             9            26               -17
210           Stanford            10             2                 8
57            Ohio St.            11            26               -15
30      North Carolina            12             3                 9
37              Dayton            13            26               -13
229       Presbyterian            

In [19]:
rankings['team']

132       South Carolina
167            Wisconsin
41                  Duke
141             NC State
166                Brown
84              Marshall
142        West Virginia
74               Gonzaga
88         Western Mich.
210             Stanford
57              Ohio St.
30        North Carolina
37                Dayton
229         Presbyterian
48             Creighton
150    Purdue Fort Wayne
113              Clemson
10             San Diego
147                 Elon
76               Vermont
65          Sacred Heart
208                 UMBC
54              Delaware
13                  Penn
233             Maryland
Name: team, dtype: object